# Set dataset parameters
Choose the attributes to load the dataset.

In [1]:
detectors = ['mtcnn']  # 'dlib', 'opencv',
embedding_models = ['ArcFace']  # "Dlib", 'VGG-Face','FaceVACs', 'Facenet512'
# TODO:add quality models to experiments
# quality_models = ['ser_fiq', 'tface']


# Set LR parameters
Calibration, image filters, face filters, quality filters...

## Calibration

In [2]:
calibrators = ['logit', 'KDE']  # 'KDE', , 'isotonic'
calibration_db = ["LFW", "SCFace"] # which datasets using to calibrate
filters = []  # todo: change name to image filters. E.g. 'pose'
face_image_filters = []  # 'confusion_score','quality'

n_calibration_pairs = 100

## Validation

In [3]:
enfsi_years = [2011, 2012]

## Other parameters
Rest of the configuration

In [4]:
metrics = "cosine"
embedding_model_as_scorer = True

# Scenarios
Different filters to do computations

In [5]:
scenario0 = []
scenario1 = ['gender', 'age']
scenario2 = ['yaw', 'pitch']
scenario3 = ['glasses', 'beard']
scenario4 = ['low_quality']
scenario5 = ['headgear']
scenario6 = ['terhorst_quality_rounded']

# Input and output directories
Where the sql database is and which folder the results are saved.

In [6]:
home = os.path.expanduser("~")

# SQL database path and name
input_dir = os.path.join(home, 'video_resources', 'sql_database')
db_name = 'colab_main_dataset'

# Where results are saved
output_dir = os.path.join(home, 'video_resources', 'exp_output')

# Preprocess data
Connect to sql database.

In [7]:
from sql_face.alchemy import get_session
from sql_face.tables import *

2022-12-12 20:37:27.558563: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 20:37:28.366661: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/andrea/anaconda3/envs/lr-video/lib/python3.8/site-packages/cv2/../../lib64:/home/andrea/Documents/cuda_software/cuda/lib64:/usr/local/cuda-11.2/lib64
2022-12-12 20:37:28.366727: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or di

In [8]:
session = get_session(input_dir, db_name)

# Start experiments

All parameters are specified in ExperimentalSetup, that creates a different experiment with all the posible combinations of detectors, embedding_models, calibrators, etc.

In [9]:
from lr_video_face.experiments import ExperimentalSetup

In [10]:
experimental_setup = ExperimentalSetup(detectors, 
                                        embedding_models, 
                                        calibrators, 
                                        calibration_db, 
                                        enfsi_years,
                                        filters, 
                                        face_image_filters, 
                                        metrics, 
                                        n_calibration_pairs,
                                        embedding_model_as_scorer,
                                        output_dir,
                                        session)

In [11]:
experimental_setup.__dict__

{'detectors': ['mtcnn'],
 'embeddingModels': ['ArcFace'],
 'calibrators': [LogitCalibrator(), KDECalibrator(bandwidth=(None, None))],
 'calibration_db': ['LFW', 'SCFace'],
 'enfsi_years': [2011, 2012],
 'filters': [],
 'face_image_filters': [],
 'metrics': 'cosine',
 'n_calibration_pairs': 100,
 'embedding_model_as_scorer': True,
 'session': <sqlalchemy.orm.session.Session at 0x7ff487e90190>,
 'name': '2022-12-12 20 37 31',
 'output_dir': '/home/andrea/video_resources/exp_output/C_(100)_[LFW,SCFace]_T_[                2011,2012]/[]',
 'experiments': [<lr_video_face.experiments.Experiment at 0x7ff48517a220>,
 'cllr_expert_per_year': defaultdict(list,
             {2011: [0.4941270736033566,
               0.5492113001744969,
               0.7026598290722597,
               0.2607763102785555,
               0.16592504925773438,
               0.363216659696734,
               0.37984499769488217,
               0.35379794879511217,
               0.3695096638501844,
               0.40